In [1]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import wordnet
from collections import  Counter
import spacy
from spacy import displacy
import enchant
from nltk.tokenize import MWETokenizer

In [2]:
df=pd.read_csv('Complete_df.csv')

# Data Handling

## Reviews Rate:
* it was amazing = 5 stars
* really liked it = 4 stars
* liked it = 3 stars
* it was ok = 2 stars
* did not like it = 1 star

In [3]:
to_rep = {'it was amazing':5,'really liked it':4, 'liked it':3, 'it was ok':2, 'did not like it':1}
df['Review_rate']=df['Review_rate'].replace(to_rep)

## Genres
According to my classification, there are 8 general genres:
* Fiction = 1
* Nonfiction = 2
* Young = 3
* Adult = 4
* Children = 5
* Short = 6
* Sequential = 7
* New = 8

In [4]:
df['Genre'].unique()

array(['Classics', 'Fantasy', 'Nonfiction', 'Historical', 'Fiction',
       'Young', 'Childrens', 'Religion', 'Science', 'Poetry', 'Horror',
       'History', 'Mystery', 'Travel', 'Romance', 'Philosophy',
       'Christian', 'Sequential', 'Academic', 'Self', 'Language', 'Plays',
       'Thriller', 'Cultural', 'Biography', 'Autobiography', 'Esoterica',
       'Drama', 'Contemporary', 'Short', 'Crime', 'Feminism', 'Adventure',
       'Food', 'Psychology', 'Holiday', 'Spirituality', 'Business',
       'Novels', 'Apocalyptic', 'Inspirational', 'Realistic', 'Economics',
       'Marriage', 'Book', 'Humor', 'Polyamorous', 'Womens', 'New',
       'Erotica', 'Politics', 'War', 'Epic', 'Parenting', 'Gender',
       'Education', 'Adult', 'Gothic', 'Paranormal', 'Animals', 'LGBT',
       'Modern', 'Magical', 'Art', 'Dark', 'Suspense', 'Shapeshifters',
       'Westerns', 'Leadership', 'How', 'Music', 'Medical', 'European',
       'Literary', 'World', 'Military', 'Sports', 'Sociology',
       'Chris

In [5]:
df['Genre'].replace(['Classics', 'Fantasy','Horror','Mystery', 'Romance','Thriller', 'Drama','Crime',
                     'Adventure', 'Novels','Erotica','Gothic', 'Paranormal','Magical','Suspense', 
                     'Shapeshifters','Westerns', 'Literary', 'Literature', ], 'Fiction', inplace=True)
df['Genre'].replace(['Historical','Religion', 'Science', 'Poetry','History','Travel','Philosophy', 
                     'Christian','Academic', 'Self', 'Language', 'Plays', 'Cultural', 'Biography',
                     'Autobiography', 'Esoterica','Contemporary','Feminism','Food', 'Psychology', 
                     'Holiday', 'Spirituality', 'Business','Apocalyptic', 'Inspirational', 
                     'Realistic', 'Economics','Marriage', 'Book', 'Humor', 'Polyamorous', 
                     'Womens', 'Politics', 'War', 'Epic', 'Gender', 'Education','Animals', 
                     'LGBT', 'Modern', 'Art', 'Parenting', 'Dark','Leadership', 'How', 'Music',
                     'Medical', 'European','World', 'Military', 'Sports', 'Sociology',
                     'Christianity', 'Criticism','Northern', 'Mental','Death', 'Race', 'Health',
                     'Anthropology'], 'Nonfiction', inplace=True)

In [6]:
df['Genre'].unique()

array(['Fiction', 'Nonfiction', 'Young', 'Childrens', 'Sequential',
       'Short', 'New', 'Adult'], dtype=object)

In [7]:
genres_dict = {'Fiction':1, 'Nonfiction':2, 'Young':3, 'Adult':4, 'Childrens':5, 'Short':6, 'Sequential':7, 'New':8}
df['Genre'].replace(genres_dict,inplace=True)

## Cover
Based on the cover types, I separated them into four groups:
* Hardcover = 1
* Paperback = 2
* Online = 3
* Audiobook = 4

In [8]:
df['Cover'].replace(['Mass Market Paperback','Library Binding', 'Unknown Binding',
                     'Trade Paperback','Paperback/Kindle'],'Paperback', inplace=True)
df['Cover'].replace(['Kindle Edition', 'ebook','Nook'],'online', inplace=True)
df['Cover'].replace(['Leather Bound', 'Board book','Slipcased Hardcover',
                    'Capa dura'],'Hardcover', inplace=True)
df['Cover'].replace(['Audio Cassette','Audio CD', 'Audible Audio',],'Audiobook', inplace=True)

In [9]:
cover_dict = {'Hardcover':1, 'Paperback':2, 'online':3, 'Audiobook':4}
df['Cover'].replace(cover_dict,inplace=True)

## Author Followers

In [10]:
df['Author_followers'].replace(',','',regex=True,inplace=True)
df['Author_followers'] = df['Author_followers'].astype(int)

## Reviews Date


In [11]:
df['Review_date'].replace(',','',regex=True,inplace=True)
# Creating 3 columns of day, month and year
day = []
month = []
year = []
for date in df['Review_date']:
    splitted_date = date.split(' ')
    month.append(splitted_date[0])
    day.append(splitted_date[1])
    year.append(splitted_date[2])
df['Review_day'] = day
df['Review_month'] = month
df['Review_year'] = year
df.drop('Review_date',axis=1,inplace=True)

# Convert the month to numbers
mon_dict = {'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}
df['Review_month'].replace(mon_dict,inplace=True)
df['Review_day'] = df['Review_day'].astype(int)
df['Review_year'] = df['Review_year'].astype(int)

# Cut the year column to 4 bins
bins=[2000,2005,2010,2015,2030]
labels=[1,2,3,4]
df['Review_year']=pd.cut(df['Review_year'],bins,labels=labels)

## Book's Year and Month


In [12]:
month_dict = {'January':1,'February':2,'March':3,'April':4,'May':5,'June':6,'July':7,'August':8,'September':9,'October':10,'November':11,'December':12}
df['Month_of_book'].replace(month_dict,inplace=True)
df['Year_of_book'] = df['Year_of_book'].astype(int)
bins=[1900,1950,2000,2005,2010,2015,2030]
labels=[1,2,3,4,5,6]
df['Year_of_book']=pd.cut(df['Year_of_book'],bins,labels=labels)

## Pages

In [13]:
df['Pages'] = df['Pages'].astype(int)

## Normalize


In [14]:
def normalize_column(values):
    min = np.min(values)
    max = np.max(values)
    norm = (values - min)/(max-min) 
    return(pd.DataFrame(norm))

In [15]:
df['Pages'] = normalize_column(df['Pages'])
df['Author_followers'] = normalize_column(df['Author_followers'])
df['Ratings_number'] = normalize_column(df['Ratings_number'])
df['Reviews_number'] = normalize_column(df['Reviews_number'])

## Data Info

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124846 entries, 0 to 124845
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   Reviews           119163 non-null  object  
 1   Reviews_link      124846 non-null  object  
 2   Review_rate       124846 non-null  int64   
 3   Ratings_number    124846 non-null  float64 
 4   Reviews_number    124846 non-null  float64 
 5   Total_rate        124846 non-null  float64 
 6   Pages             124846 non-null  float64 
 7   Year_of_book      124816 non-null  category
 8   Month_of_book     124846 non-null  int64   
 9   Author_followers  124846 non-null  float64 
 10  Cover             124846 non-null  int64   
 11  Genre             124846 non-null  int64   
 12  Review_day        124846 non-null  int32   
 13  Review_month      124846 non-null  int64   
 14  Review_year       124846 non-null  category
dtypes: category(2), float64(5), int32(1), int64(5), obj

# NLP

In [17]:
# Drop the rows that have no review
df.drop(df[df['Reviews'].isnull()].index,axis=0,inplace=True)

In [18]:
nlp = spacy.load('en_core_web_sm')
Persons = []
for rev in df['Reviews']:
    Persons_Count = 0
    n = nlp(rev).ents
    for x in n:
        if x.label_ == 'PERSON':
            Persons_Count += 1
    Persons.append(Persons_Count)
df['Num_of_names'] = Persons

In [19]:
# Define function to lemmatize each word with its POS tag
 
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:         
        return None

## Tags:
* Noun = 'n'
* Verb = 'v'
* Adverb = 'r'
* Adjective = 'a'

In [39]:
mwe = MWETokenizer([('i', 'love','this','book'),('i', 'loved','this','book'),('loved','this','book'),('love','this','book'), ('i', 'hate','this','book'),('one', 'of', 'the','worst','books'),('one', 'of', 'the','best','books'),('worst','book'),('best','book'),('not','good')], separator='_')
reviews = []
review_length = []
Nouns = []
Adj = []
Adv = []
Ver = []
Others = []

for rev in df['Reviews']:
    rev.replace('\n','')
    # split into words
    tokens = word_tokenize(rev)
    #remove all tokens that are not english alphabetic and lower them
    d = enchant.Dict("en_US")
    All_words = [word.lower() for word in tokens if d.check(word) and word.isalpha()]
    #count words
    review_length.append(len(All_words))
    # join to a new review
    updated_review = ' '.join(All_words) 
    new_tokens = mwe.tokenize(word_tokenize(updated_review))
    # filter out stop words
    stop_words = stopwords.words('english')
    stop_words.extend(['image', 'error'])
    words = [w for w in new_tokens if w not in stop_words]
    # counting the tag types
    noun = 0
    verb = 0
    adj = 0
    adv = 0
    other = 0
    tagged = nltk.pos_tag(words)
    for (word, tag) in tagged:
        if pos_tagger(tag) == 'n':
            noun +=1
        elif pos_tagger(tag) == 'v':
            verb +=1
        if pos_tagger(tag) == 'a':
            adj +=1
        if pos_tagger(tag) == 'r':
            adv +=1 
        else:
            other +=1
    #lemmitizing of words
    lemmitizer = WordNetLemmatizer()
    lemma = []
    words_tags = nltk.pos_tag(words)
    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), words_tags))
    for (word, tag) in wordnet_tagged:
            if tag is None:
                lemma.append(lemmitizer.lemmatize(word))
            else:
                lemma.append(lemmitizer.lemmatize(word,tag))

    
    if len(lemma)==0:
        reviews.append(np.nan)
    else:
        reviews.append(lemma)
    Nouns.append(noun)
    Adj.append(adj)
    Adv.append(adv)
    Ver.append(verb)
    Others.append(other)

In [40]:
df['Tokenized'] = reviews
df['Words_amount'] = review_length
df['Noun'] = Nouns
df['Adjective'] = Adj
df['Adverb'] = Adv
df['Verb'] = Ver
df['Other_words'] = Others

In [52]:
df.drop(df[df['Tokenized'].isnull()].index,axis=0,inplace=True)

In [53]:
updated_rev = []
for sen in df['Tokenized']:
    rev=''
    for w in sen:
        rev+=w+' '
    updated_rev.append(rev)
df['Updated_Review'] = updated_rev

In [101]:
# Creating the target column
Like = []
for rate in df['Review_rate']:
    if rate<=3:
        Like.append(0)
    if rate>3:
        Like.append(1)
df['Book_like'] = Like
df['Book_like'].value_counts()

1    86858
0    32044
Name: Book_like, dtype: int64

In [64]:
# For ML
df_c = df.drop(df[df['Book_like'] == 1].sample(n=55000, random_state=1).index)
df_c['Book_like'].value_counts()

0    32044
1    31858
Name: Book_like, dtype: int64

In [92]:
positive_words = ['loved','love','good','amaze','amazed','excellent','amazing','great','wonderful','beautiful','fantastic','incredible']
negative_words = ['evil','horrible','hate','awful','bad','lack','lacks','low','stupid','disappoint','boring','bore','blah','fuck','dislike','disliked','terrible']
pos = []
neg = []
for rev in df_c['Tokenized']:
    pos_count = 0
    neg_count = 0
    for word in rev:
        if word in negative_words:
            neg_count += 1
        elif word in positive_words:
            pos_count += 1
    pos.append(pos_count)
    neg.append(neg_count)
df_c['Positive'] = pos
df_c['Negative'] = neg

In [78]:
group0 = 'not_good'
group1 = 'i_love_this_book'
group2 = 'i_loved_this_book'
group3 = 'love_this_book'
group4 = 'loved_this_book'
group5 = 'i_hate_this_book'
group6 = 'one_of_the_worst_books'
group7 = 'one_of_the_best_books'
group8 = 'worst_book'
group9 = 'best_book'
l0 = []
l1 = []
l2 = []
l3 = []
l4 = []
l5 = []
l6 = []
l7 = []
l8 = []
l9 = []

for rev in df_c['Tokenized']:
    c0 = c1 = c2 = c3 = c4 = c5 = c6 = c7 = c8 = c9 = 0
    for word in rev:
        if word==group0:
            c0+=1
        elif word==group1:
            c1+=1
        elif word==group2:
            c2+=1
        elif word==group3:
            c3+=1
        elif word==group4:
            c4+=1
        elif word==group5:
            c5+=1
        elif word==group6:
            c6+=1
        elif word==group7:
            c7+=1
        elif word==group8:
            c8+=1
        elif word==group9:
            c9+=1
    
    l0.append(c0) 
    l1.append(c1)
    l2.append(c2) 
    l3.append(c3)
    l4.append(c4) 
    l5.append(c5)
    l6.append(c6) 
    l7.append(c7)
    l8.append(c8) 
    l9.append(c9)

df_c['not_good'] = l0 
df_c['i_love_this_book'] = l1
df_c['i_loved_this_book'] = l2
df_c['love_this_book'] = l3
df_c['loved_this_book'] = l4
df_c['i_hate_this_book'] = l5
df_c['one_of_the_worst_books'] = l6
df_c['one_of_the_best_books'] = l7
df_c['worst_book'] = l8
df_c['best_book'] = l9

In [93]:
df_c.dropna(inplace=True)

In [94]:
df_c.to_csv('DATA_After_Cleaning.csv', index = False)

In [95]:
df_c.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63890 entries, 0 to 124845
Data columns (total 37 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   Reviews                 63890 non-null  object  
 1   Reviews_link            63890 non-null  object  
 2   Review_rate             63890 non-null  int64   
 3   Ratings_number          63890 non-null  float64 
 4   Reviews_number          63890 non-null  float64 
 5   Total_rate              63890 non-null  float64 
 6   Pages                   63890 non-null  float64 
 7   Year_of_book            63890 non-null  category
 8   Month_of_book           63890 non-null  int64   
 9   Author_followers        63890 non-null  float64 
 10  Cover                   63890 non-null  int64   
 11  Genre                   63890 non-null  int64   
 12  Review_day              63890 non-null  int32   
 13  Review_month            63890 non-null  int64   
 14  Review_year          